In [1]:
import pandas as pd
import numpy as np

In [19]:
years = [2022, 2023]
new_russian_team = np.array([1083, 2006])
new_england_team = np.array([1359, 44, 62])
new_bundes_team = np.array([180, 181])
new_serie_team = np.array([512, 490, 495])

#Россия
all_dataRFPL = pd.DataFrame()
for year in years:
    rfpl_path = f'elo/csv/Russia/{year}/RFPL.csv'
    kubok_path = f'elo/csv/Russia/{year}/KUBOK.csv'

    data_RFPL = pd.read_csv(rfpl_path)
    data_kubok = pd.read_csv(kubok_path)
    if year == 2022:
        fnl_path = f'elo/csv/Russia/2022/FNL.csv'
        data_fnl = pd.read_csv(fnl_path)
        data_fnl = data_fnl[data_fnl['home_team_id'].isin(new_russian_team) | data_fnl['away_team_id'].isin(new_russian_team)]
        unique_teams_RFPL = np.concatenate((data_RFPL['home_team_id'].unique(), new_russian_team))
    else:
        unique_teams_RFPL = data_RFPL['home_team_id'].unique()
        data_fnl = pd.DataFrame()

    data_kubok = data_kubok[data_kubok['home_team_id'].isin(unique_teams_RFPL) & data_kubok['away_team_id'].isin(unique_teams_RFPL)]
    all_dataRFPL = pd.concat([all_dataRFPL, data_kubok, data_RFPL, data_fnl])

print(len(all_dataRFPL))

555


In [20]:
all_dataRFPL = all_dataRFPL.sort_values('timestamp')
all_dataRFPL.reset_index(drop=True, inplace=True)


In [21]:
#Англия
all_dataPL = pd.DataFrame()
for year in years:
    pl_path = f'elo/csv/England/{year}/PL.csv'
    kubok_path = f'elo/csv/England/{year}/cup.csv'
    carabao_path = f'elo/csv/England/{year}/carabao.csv'

    data_PL = pd.read_csv(pl_path)
    data_kubok = pd.read_csv(kubok_path)
    data_carabao = pd.read_csv(carabao_path)

    if year == 2022:
        championship_path = f'elo/csv/England/2022/championship.csv'
        data_championship = pd.read_csv(championship_path)
        data_championship = data_championship[data_championship['home_team_id'].isin(new_england_team) | data_championship['away_team_id'].isin(new_england_team)]
        unique_teams_PL = np.concatenate((data_PL['home_team_id'].unique(), new_england_team))
    else:
        unique_teams_PL = data_PL['home_team_id'].unique()
        data_championship = pd.DataFrame()

    data_kubok = data_kubok[data_kubok['home_team_id'].isin(unique_teams_PL) & data_kubok['away_team_id'].isin(unique_teams_PL)]
    data_carabao = data_carabao[data_carabao['home_team_id'].isin(unique_teams_PL) & data_carabao['away_team_id'].isin(unique_teams_PL)]
    all_dataPL = pd.concat([all_dataPL, data_kubok, data_PL, data_carabao, data_championship])

all_dataPL = all_dataPL.sort_values('timestamp')
all_dataPL.reset_index(drop=True, inplace=True)
all_dataPL

,fixtueID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
0,874671,1659121200,37,Huddersfield,0.0,1.0,44,Burnley
1,874676,1659189600,1359,Luton,0.0,0.0,54,Birmingham
2,874682,1659380400,38,Watford,1.0,0.0,62,Sheffield Utd
3,867946,1659726000,52,Crystal Palace,0.0,2.0,42,Arsenal
4,867947,1659785400,36,Fulham,2.0,2.0,40,Liverpool
...,...,...,...,...,...,...,...,...
686,1035300,1700924400,34,Newcastle,4.0,1.0,49,Chelsea
687,1035294,1700933400,55,Brentford,0.0,1.0,42,Arsenal
688,1035303,1701007200,47,Tottenham,1.0,2.0,66,Aston Villa
689,1035296,1701016200,45,Everton,0.0,3.0,33,Manchester United


In [22]:
all_data = pd.concat([all_dataPL, all_dataRFPL]).sort_values('timestamp')
all_data.reset_index(drop=True, inplace=True)

In [23]:
all_data

,fixtueID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
0,879867,1657904400,1994,Khimki,1.0,1.0,596,Zenit Saint Petersburg
1,879868,1657972800,555,CSKA Moscow,2.0,0.0,1084,Ural
2,879869,1657981800,1080,Orenburg,2.0,4.0,1079,Krylya Sovetov
3,879870,1657990800,1085,Akhmat Grozny,1.0,1.0,558,Spartak Moscow
4,881315,1658055600,6833,Veles,2.0,1.0,2006,Baltika
...,...,...,...,...,...,...,...,...
1241,1035297,1701115200,36,Fulham,3.0,2.0,39,Wolves
1242,1141323,1701184500,2006,Baltika,2.0,2.0,597,Lokomotiv Moscow
1243,1141319,1701262800,1080,Orenburg,1.0,0.0,558,Spartak Moscow
1244,1141325,1701270900,779,FC Rostov,1.0,1.0,555,CSKA Moscow


In [7]:
import http.client
import json

conn = http.client.HTTPSConnection("v3.football.api-sports.io")

headers = {
    'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': "55d8d0822bdb3026f52b479e85e3a447"
    }

last_time_download = 1699806600
result_data = []
for index, row in all_data.iterrows():
    fixtureID = row['fixtueID']
    timestamp = row['timestamp']
    home_team_id = row['home_team_id']
    home_goals = row['home_goals']
    away_team_id = row['away_team_id']
    away_goals = row['away_goals']
    
    if (row['timestamp'] > last_time_download):
        for team_info, position in [((home_team_id, home_goals), 'home'), ((away_team_id, away_goals), 'away')]:
            conn.request("GET", f"/fixtures/statistics?fixture={fixtureID}&team={team_info[0]}", headers=headers)
            res = conn.getresponse()
            data = res.read()
            json_data = json.loads(data)

            if 'response' in json_data and json_data['response']:
                json_data = json_data['response']
                if json_data and len(json_data) > 0 and 'statistics' in json_data[0]:
                    stat = json_data[0]['statistics']
                else:
                    stat = []
            else:
                stat = []

            if len(stat) > 0:
                # Получение ударов в створ и XG
                shots_on_target = [item['value'] for item in stat if item['type'] == 'Shots on Goal']
                if shots_on_target and isinstance(shots_on_target[0], int):
                    shots_on_target = shots_on_target[0]
                else:
                    shots_on_target = team_info[1]

                xg = [item['value'] for item in stat if item['type'] == 'expected_goals']
                if xg and isinstance(xg[0], str):
                    xg = xg[0]
                else:
                    xg = team_info[1]

                result_data.append([fixtureID, team_info[0], position, team_info[1], shots_on_target, xg, timestamp])

result_df = pd.DataFrame(result_data, columns=['fixtureID', 'team_id', 'position', 'goals', 'shots', 'xg', 'time'])

print(result_df)

    

    fixtureID  team_id position  goals  shots    xg        time
0     1042136     1079     home    3.0   10.0   3.0  1700906400
1     1042136      597     away    3.0    4.0   3.0  1700906400
2     1042134      596     home    3.0    7.0   3.0  1700914500
3     1042134     2012     away    0.0    1.0   0.0  1700914500
4     1035299       50     home    1.0    5.0  1.36  1700915400
5     1035299       40     away    1.0    3.0  0.56  1700915400
6     1042137      555     home    2.0    6.0   2.0  1700922600
7     1042137     1088     away    3.0    6.0   3.0  1700922600
8     1035298     1359     home    2.0    3.0  0.94  1700924400
9     1035298       52     away    1.0    8.0  1.67  1700924400
10    1035300       34     home    4.0    5.0  2.36  1700924400
11    1035300       49     away    1.0    4.0  0.53  1700924400
12    1035301       65     home    2.0    5.0  1.93  1700924400
13    1035301       51     away    3.0    5.0  1.48  1700924400
14    1035302       62     home    1.0  

In [8]:
result_df['xg'] = result_df['xg'].astype('float')
result_df.to_csv('match_data.csv',float_format='%.2f', mode='a')

In [16]:
#Germany
all_dataBundes = pd.DataFrame()
for year in years:
    bundes_path = f'elo/csv/Germany/{year}/Bundes.csv'
    kubok_path = f'elo/csv/Germany/{year}/DFB.csv'

    data_bundes = pd.read_csv(bundes_path)
    data_kubok = pd.read_csv(kubok_path)
    if year == 2022:
        bundes2_path = f'elo/csv/Germany/2022/Bundes2.csv'
        data_bundes2 = pd.read_csv(bundes2_path)
        data_bundes2 = data_bundes2[data_bundes2['home_team_id'].isin(new_bundes_team) | data_bundes2['away_team_id'].isin(new_bundes_team)]
        unique_teams_Bundes = np.concatenate((data_bundes['home_team_id'].unique(), new_bundes_team))
    else:
        unique_teams_Bundes = data_bundes['home_team_id'].unique()
        data_bundes2 = pd.DataFrame()

    data_kubok = data_kubok[data_kubok['home_team_id'].isin(unique_teams_Bundes) & data_kubok['away_team_id'].isin(unique_teams_Bundes)]
    all_dataBundes = pd.concat([all_dataBundes, data_kubok, data_bundes, data_bundes2])

print(len(all_dataBundes))
all_dataBundes

502


,fixtueID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
39,958525,1666108800,173,RB Leipzig,4,0,175,Hamburger SV
40,958526,1666205100,170,FC Augsburg,2,5,157,Bayern Munich
42,958528,1666118700,167,1899 Hoffenheim,5,1,174,FC Schalke 04
45,958531,1666205100,182,Union Berlin,2,0,180,FC Heidenheim
47,958533,1666118700,181,SV Darmstadt 98,2,1,163,Borussia Monchengladbach
...,...,...,...,...,...,...,...,...
103,1048984,1700922600,161,VfL Wolfsburg,2,1,173,RB Leipzig
104,1048985,1700854200,192,FC Koln,0,1,157,Bayern Munich
105,1048986,1701016200,167,1899 Hoffenheim,1,1,164,FSV Mainz 05
106,1048987,1700922600,162,Werder Bremen,0,3,168,Bayer Leverkusen


In [14]:
#Italt
all_dataItaly = pd.DataFrame()
for year in years:
    bundes_path = f'elo/csv/Italy/{year}/SeriaA.csv'
    kubok_path = f'elo/csv/Italy/{year}/KUBOK.csv'

    data_bundes = pd.read_csv(bundes_path)
    data_kubok = pd.read_csv(kubok_path)
    if year == 2022:
        bundes2_path = f'elo/csv/Italy/2022/SeriaB.csv'
        data_bundes2 = pd.read_csv(bundes2_path)
        data_bundes2 = data_bundes2[data_bundes2['home_team_id'].isin(new_serie_team) | data_bundes2['away_team_id'].isin(new_serie_team)]
        unique_teams_Bundes = np.concatenate((data_bundes['home_team_id'].unique(), new_serie_team))
    else:
        unique_teams_Bundes = data_bundes['home_team_id'].unique()
        data_bundes2 = pd.DataFrame()

    data_kubok = data_kubok[data_kubok['home_team_id'].isin(unique_teams_Bundes) & data_kubok['away_team_id'].isin(unique_teams_Bundes)]
    all_dataItaly = pd.concat([all_dataItaly, data_kubok, data_bundes, data_bundes2])

print(len(all_dataItaly))

646


In [15]:
all_dataItaly

,fixtueID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
8,897497,1659899700,1579,Monza,3,2,512,Frosinone
24,964064,1666206000,494,Udinese,2,3,1579,Monza
27,964067,1666292400,500,Bologna,1,0,490,Cagliari
28,967210,1673467200,489,AC Milan,0,1,503,Torino
30,967212,1674136800,499,Atalanta,5,2,515,Spezia
...,...,...,...,...,...,...,...,...
125,1052378,1701106200,504,Verona,2,2,867,Lecce
126,1052379,1701027900,496,Juventus,1,1,505,Inter
127,1052380,1700941500,489,AC Milan,1,0,502,Fiorentina
128,1052381,1701018000,497,AS Roma,3,1,494,Udinese


In [24]:
all_data = pd.concat([all_dataPL, all_dataRFPL, all_dataBundes, all_dataItaly]).sort_values('timestamp')
all_data.reset_index(drop=True, inplace=True)

In [25]:
all_data

,fixtueID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
0,879867,1657904400,1994,Khimki,1.0,1.0,596,Zenit Saint Petersburg
1,872868,1657969200,177,Jahn Regensburg,2.0,0.0,181,SV Darmstadt 98
2,879868,1657972800,555,CSKA Moscow,2.0,0.0,1084,Ural
3,879869,1657981800,1080,Orenburg,2.0,4.0,1079,Krylya Sovetov
4,879870,1657990800,1085,Akhmat Grozny,1.0,1.0,558,Spartak Moscow
...,...,...,...,...,...,...,...,...
2389,1035297,1701115200,36,Fulham,3.0,2.0,39,Wolves
2390,1141323,1701184500,2006,Baltika,2.0,2.0,597,Lokomotiv Moscow
2391,1141319,1701262800,1080,Orenburg,1.0,0.0,558,Spartak Moscow
2392,1141321,1701270900,1088,Dinamo Moscow,1.0,0.0,596,Zenit Saint Petersburg


In [26]:
import http.client
import json

conn = http.client.HTTPSConnection("v3.football.api-sports.io")

headers = {
    'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': "55d8d0822bdb3026f52b479e85e3a447"
    }

last_time_download = -1
result_data = []
for index, row in all_data.iterrows():
    fixtureID = row['fixtueID']
    timestamp = row['timestamp']
    home_team_id = row['home_team_id']
    home_goals = row['home_goals']
    away_team_id = row['away_team_id']
    away_goals = row['away_goals']
    
    if (row['timestamp'] > last_time_download):
        for team_info, position in [((home_team_id, home_goals), 'home'), ((away_team_id, away_goals), 'away')]:
            conn.request("GET", f"/fixtures/statistics?fixture={fixtureID}&team={team_info[0]}", headers=headers)
            res = conn.getresponse()
            data = res.read()
            json_data = json.loads(data)

            if 'response' in json_data and json_data['response']:
                json_data = json_data['response']
                if json_data and len(json_data) > 0 and 'statistics' in json_data[0]:
                    stat = json_data[0]['statistics']
                else:
                    stat = []
            else:
                stat = []

            if len(stat) > 0:
                # Получение ударов в створ и XG
                shots_on_target = [item['value'] for item in stat if item['type'] == 'Shots on Goal']
                if shots_on_target and isinstance(shots_on_target[0], int):
                    shots_on_target = shots_on_target[0]
                else:
                    shots_on_target = team_info[1]

                xg = [item['value'] for item in stat if item['type'] == 'expected_goals']
                if xg and isinstance(xg[0], str):
                    xg = xg[0]
                else:
                    xg = team_info[1]

                result_data.append([fixtureID, team_info[0], position, team_info[1], shots_on_target, xg, timestamp])

result_df = pd.DataFrame(result_data, columns=['fixtureID', 'team_id', 'position', 'goals', 'shots', 'xg', 'time'])

print(result_df)

      fixtureID  team_id position  goals  shots   xg        time
0        879867     1994     home    1.0    3.0  1.0  1657904400
1        879867      596     away    1.0    2.0  1.0  1657904400
2        872868      177     home    2.0    4.0  2.0  1657969200
3        872868      181     away    0.0    3.0  0.0  1657969200
4        879868      555     home    2.0    8.0  2.0  1657972800
...         ...      ...      ...    ...    ...  ...         ...
4643    1141319      558     away    0.0    1.0  0.0  1701262800
4644    1141321     1088     home    1.0    3.0  1.0  1701270900
4645    1141321      596     away    0.0    0.0  0.0  1701270900
4646    1141325      779     home    1.0    3.0  1.0  1701270900
4647    1141325      555     away    1.0    2.0  1.0  1701270900

[4648 rows x 7 columns]


In [27]:
result_df['xg'] = result_df['xg'].astype('float')
result_df.to_csv('match_data.csv',float_format='%.2f')